##### Target URLs

In [1]:
# !pip install -U -q "google-genai"

In [2]:
# target urls
text_urls = """
https://www.foxnews.com/politics/hirono-ripped-for-opening-confirmation-hearing-question-to-burgum-this-lady-has-issues
https://www.foxnews.com/world/brazilian-court-denies-bolsonaros-request-travel-trumps-inauguration
https://www.foxnews.com/world/american-college-rape-suspect-being-extradited-us

https://www.foxnews.com/politics/bidens-meta-censorship-revelations-vindicated-alito-dissent-rfk-jr-case
https://www.foxnews.com/politics/bondi-spars-schiff-testy-confirmation-hearing-you-were-censured
https://www.foxnews.com/us/starship-upper-stage-lost-seventh-test-flight-debris-seen-spewing-sky

https://www.breitbart.com/clips/2025/01/16/biden-i-ignored-summers-inflation-warnings-to-instill-some-confidence-in-people-deal-with-covid/
https://www.breitbart.com/clips/2025/01/16/white-house-we-didnt-get-inflation-spike-with-first-trump-tariffs-but-25-different-from-17/
https://www.breitbart.com/politics/2025/01/16/trump-treasury-pick-disputes-democrat-assertion-america-is-in-clean-energy-arms-race-with-china/
"""


# helper

In [3]:
# get current_date_time
from datetime import datetime

# create document tile (date_time)
def get_doc_title():
  current_date_time = datetime.now()
  # print(current_date_time)

  # concat date_time to string
  current_date_time_str = current_date_time.strftime("%Y%m%d_%H%M%S")
  # print(current_date_time_str)
  return "textbook_" + current_date_time_str


def print_first_last_lines(text):
  lines = text.split("\n")

  for line in lines[:5]:
    print(line)
  print("........")
  for line in lines[-5:]:
    print(line)

# create url list[str]
def read_url_text(text_urls):
  urls = text_urls.split("\n")

  # remove null or empty element in the urls
  urls = [url.strip() for url in urls if url.strip()]

  # remove space
  urls = [url.strip() for url in urls]
  return urls


# Init GenAI

In [4]:
from google import genai
from google.genai import types
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Explicitly configure the API key using genai.configure
# genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))



client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))




# Contents generation


In [5]:
def generate_ai_text(prompt, contents):

  contents = prompt + "\n\n" + contents

  response = client.models.generate_content(
      model='gemini-1.5-flash', contents=contents
  )
  return response.text

# Fox news extraction function

In [6]:
import requests
from bs4 import BeautifulSoup

def extract_fox_news(url):

  print(url)

  return_text = ""

    # 웹 페이지 요청
  response = requests.get(url, verify=True)

  # 페이지의 HTML 콘텐츠 파싱
  soup = BeautifulSoup(response.content, 'html.parser')

  # Get title
  title = soup.find_all('title')
  # print(f"title: {title[0].get_text()}")
  # print(f"{title[0].get_text()}")
  # print(title)
  return_text = title[0].get_text() + "\n"

  #Get sub_headline
  sub_headline = soup.find_all("h2", class_="sub-headline")
  # print(f"sub headline : {sub_headline[0].get_text()}")
  # print(f"{sub_headline[0].get_text()}")
  # print(f"sub_headline = {sub_headline}")
  return_text += sub_headline[0].get_text() + "\n"



  # 예시: 헤드라인 뉴스 크롤링
  headlines = soup.find_all('div', class_='article-body')

  for headline in headlines:
      # print_sentences_with_period(headline.get_text())
      # print (headline.get_text())
      return_text += headline.get_text() + "\n"

  return return_text

# Beitbart news extraction function

In [7]:
import requests
from bs4 import BeautifulSoup

def extract_breitbart_news(url):

  print(url)

  return_text = ""

  # 웹 페이지 요청
  response = requests.get(url, verify=True)

  # 페이지의 HTML 콘텐츠 파싱
  soup = BeautifulSoup(response.content, 'html.parser')

  # Get title
  title = soup.find_all('title')
  # print(f"{title[0].get_text()}")

  return_text = title[0].get_text() + "\n"

  #Get sub_headline
  # sub_headline = soup.find_all("p", class_="subheading")
  # print(f"sub headline : {sub_headline[0].get_text()}")
  # for text in sub_headline:
    # print(text.get_text())

  # print("\n")
  # print(f"{sub_headline[0].get_text()}")


  # 예시: 헤드라인 뉴스 크롤링
  headlines = soup.find_all('p')

  for headline in headlines:
      # print_sentences_with_period(headline.get_text())
      # print (headline.get_text())
      return_text += headline.get_text() + "\n"

  return return_text

# Main

In [8]:
FOXNEWS = "foxnews"
BREIBART = "breitbart"

# output dictionary
file_output = ""

# get urls
urls = read_url_text(text_urls)

for url in urls:

  #file output - copy url
  file_output += url + "\n\n"

  if FOXNEWS in url:
    text = extract_fox_news(url)
  elif BREIBART in url:
    text = extract_breitbart_news(url)
  else:
    print("Error : New Link is not supported yet")

  query = """
     Devide the text into sentences.
     For each sentences, do next action items print the output with below format:
     - English sentence
     - List up IELT’s B2/C1/C2 vocabulrary with korean translation, without separate the words with newline.
     - sentence Korean translation
    """

  query = """
  다음 내용에서 사용된 영어 중급 레벨 이상의 "phrasal verbs"을 찾아줘. 전치사 없이 동사만 사용된 경우는 제외해줘.

  출력 형식은 다음과 같이 해줘.
  - phrasal verb (한글뜻) / 문장속의 예문
  """

  output = generate_ai_text(query,text)
  print(output)

  #file output - copy output
  file_output += output + "\n"

  query = """
다음 영어 텍스트를 분석하여 문장 단위로 나누고, 각 문장에 대해 다음 정보를 각 변수에 담아줘.

1. 영어 문장 (Original English Sentence) -> 변수 : original_english_sentence
2. IELT의 B2/C1/C2 수준의 단어/표현과 한글 뜻 : 변수 : key_words_and_korean_definitions
3. 번역 (Sentence translation in Korean): 변수 : sentence_translation_in_korean

출력 형식은 다음과 같이 해주고, 변수 이름은 출력하지 말고 변수값만 출력.

original_english_sentence

- key_words_and_korean_definitions
-  sentence_translation_in_korean
"""

  output = generate_ai_text(query,text)
  # print(output)
  print_first_last_lines(output)

  #file output - copy output
  file_output += output + "\n"

file_output += "[End of Document]"

https://www.foxnews.com/politics/hirono-ripped-for-opening-confirmation-hearing-question-to-burgum-this-lady-has-issues
- **claps back** (반박하다) / BONDI CLAPS BACK AT HIRONO AMID QUESTIONS ABOUT POLITICAL PROSECUTIONS: 'REFUSED TO MEET WITH ME'
- **stands by** (지지하다, 변호하다) / DEM SENATOR WHO BASHED HEGSETH'S QUALIFICATIONS STANDS BY DOD SEC WHO OVERSAW BOTCHED AFGHAN WITHDRAWAL
- **panned** (혹평하다) / ...was widely panned by conservatives on social media over the line of questioning.
- **settles related to** (…와 관련된 합의를 하다) / "Have you ever faced discipline or entered into a settlement related to this kind of conduct?"
- **sit on** (위원회 등에 참여하다) / ...before any of the committees on which I sit,...
- **rips for** (~을 맹렬히 비난하다) / Hirono ripped for 'deranged' opening confirmation hearing question to Burgum: 'This lady has issues'
- **asks for** (~을 요청하다) / Have you ever made unwanted requests for sexual favors



Hirono ripped for 'deranged' opening confirmation hearing question to Burgum: 'T

# save to file - google doc

In [9]:
from google.colab import auth
# auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

from googleapiclient.http import MediaFileUpload

from google.oauth2 import service_account

In [10]:
from googleapiclient.discovery import build

def create_doc(title, content, folder_id=None):
    docs_service = build('docs', 'v1')

    document = docs_service.documents().create(body={'title': title}).execute()
    doc_id = document.get('documentId')

    requests = []
    requests.append({
        'insertText': {
            'location': {
                'index': 1,
            },
            'text': content
        }
    })

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

    # 폴더 지정
    if folder_id:
      file_service = build('drive', 'v3')
      file_service.files().update(fileId=doc_id, addParents=folder_id).execute()

    print(f'Google Docs 파일이 생성되었습니다. ID: {doc_id}')
    return doc_id


text_content = file_output
doc_title = get_doc_title()
folder_id = '1xEcPyhD5K727rmK7ePL0vJneh118YVqK'  # Google Drive 폴더 ID를 넣어주세요

document_id = create_doc(doc_title, text_content, folder_id)

Google Docs 파일이 생성되었습니다. ID: 1Pl4ynqj7_QdBuoQQO8aBqDOw5tBqV-UhFeDjSRIcP1s


*# End of Project*

In [11]:
text_urls = """
https://www.foxnews.com/live-news/israel-hamas-cease-fire-hostage-release-live-updates-1-16-2025
https://www.foxnews.com/world/brazilian-court-denies-bolsonaros-request-travel-trumps-inauguration
https://www.foxnews.com/world/american-college-rape-suspect-being-extradited-us
"""
# read string line by line and save to list
def read_url_text(text_urls):
  urls = text_urls.split("\n")

  # remove null or empty element in the urls
  urls = [url.strip() for url in urls if url.strip()]

  urls = [url.strip() for url in urls]
  return urls

urls = read_url_text(text_urls)
for url in urls:
  print(f"value = {url}")





value = https://www.foxnews.com/live-news/israel-hamas-cease-fire-hostage-release-live-updates-1-16-2025
value = https://www.foxnews.com/world/brazilian-court-denies-bolsonaros-request-travel-trumps-inauguration
value = https://www.foxnews.com/world/american-college-rape-suspect-being-extradited-us


In [12]:
text = """
Trump's UN ambassador pick Elise Stefanik could save taxpayers millions if taps Musk-Ramaswamy 'DOGE' | Fox News

- ambassador(대사), pick(선택), taxpayers(납세자), millions(수백만), taps(활용하다), Musk(머스크), Ramaswamy(라마스와미), DOGE(Dogecoin, 암호화폐), save(절약하다)
- 트럼프의 유엔 대사 후보인 엘리스 스테파닉이 머스크와 라마스와미의 'DOGE'를 활용한다면 납세자 수백만 달러를 절약할 수 있다는 기사입니다.


The United Nations is often working against US interests, critics claim

- critics(비평가들), claim(주장하다), interests(이익)
- 비평가들은 유엔이 종종 미국의 이익에 반하는 활동을 하고 있다고 주장합니다.


close      Video UN Watch exec director 'confident' in Elise Stefanik's ability to serve as UN ambassador  U.N. Watch Executive Director Hillel Neuer discusses the impact of the president-elect's appointments, the 'highjacking' of the U.N., the Biden administration's handling of the Israel war and Iran debating whether to make a deal with Trump. Join Fox News for access to this content Plus special access to select articles and other premium content with your account - free of charge.

- executive director(상무이사), confident(자신감 있는), ability(능력), appointments(임명), highjacking(장악), handling(처리), debating(논의하다), access(접근)
- UN 감시 단체의 상무이사인 Hillel Neuer는 차기 대통령의 임명, 유엔의 '장악', 바이든 행정부의 이스라엘 전쟁 처리, 그리고 이란이 트럼프와의 거래 여부를 논의하는 것에 대한 영향을 논의합니다. 폭스 뉴스에 가입하여 이 콘텐츠에 접근하십시오. 계정으로 특별히 선택된 기사 및 기타 프리미엄 콘텐츠에 무료로 접근할 수 있습니다.


By entering your email and pushing continue, you are agreeing to Fox News' Terms of Use and Privacy Policy, which includes our Notice of Financial Incentive.

- Terms of Use(이용 약관), Privacy Policy(개인정보 보호정책), Notice of Financial Incentive(재정적 인센티브 고지)
- 이메일을 입력하고 계속을 누르면 폭스 뉴스의 이용 약관 및 개인정보 보호정책(재정적 인센티브 고지 포함)에 동의하는 것입니다.


Please enter a valid email address.  By entering your email and pushing continue, you are agreeing to Fox News' Terms of Use and Privacy Policy, which includes our Notice of Financial Incentive.

- valid(유효한), email address(이메일 주소)
- 유효한 이메일 주소를 입력하십시오. 이메일을 입력하고 계속을 누르면 폭스 뉴스의 이용 약관 및 개인정보 보호정책(재정적 인센티브 고지 포함)에 동의하는 것입니다.


Having trouble? Click here. Rep. Elise Stefanik, R-NY., will soon appear before the Senate Foreign Relations Committee to seek confirmation for her role in President-elect Donald Trump’s Cabinet as the U.S. ambassador to the United Nations.

- confirmation(확정), role(역할), Cabinet(내각), ambassador(대사)
- 문제가 있습니까? 여기를 클릭하십시오. 뉴욕주 출신의 엘리스 스테파닉 하원의원은 곧 상원 외교 관계 위원회에 출석하여 차기 대통령인 도널드 트럼프의 내각에서 미국 유엔 대사 역할에 대한 확정을 추구할 것입니다.


Several former diplomats who spoke to Fox News Digital say that an immediate concern for an incoming U.N. ambassador should be reigning in U.S. expenditures at the world body.

- diplomats(외교관), incoming(들어오는), expenditures(지출), reigning in(억제하다), world body(국제기구)
- 폭스 뉴스 디지털과 이야기한 여러 전직 외교관들은 새롭게 유엔 대사가 되는 사람에게 있어 즉각적인 우려 사항은 세계 기구에서 미국의 지출을 억제하는 것이어야 한다고 말합니다.
"""

# print first 5 lines and last 5 lines of text
def print_first_last_lines(text):
  lines = text.split("\n")

  for line in lines[:5]:
    print(line)
  print("........")
  for line in lines[-5:]:
    print(line)

print_first_last_lines(text)


Trump's UN ambassador pick Elise Stefanik could save taxpayers millions if taps Musk-Ramaswamy 'DOGE' | Fox News

- ambassador(대사), pick(선택), taxpayers(납세자), millions(수백만), taps(활용하다), Musk(머스크), Ramaswamy(라마스와미), DOGE(Dogecoin, 암호화폐), save(절약하다)
- 트럼프의 유엔 대사 후보인 엘리스 스테파닉이 머스크와 라마스와미의 'DOGE'를 활용한다면 납세자 수백만 달러를 절약할 수 있다는 기사입니다.
........
Several former diplomats who spoke to Fox News Digital say that an immediate concern for an incoming U.N. ambassador should be reigning in U.S. expenditures at the world body.

- diplomats(외교관), incoming(들어오는), expenditures(지출), reigning in(억제하다), world body(국제기구)
- 폭스 뉴스 디지털과 이야기한 여러 전직 외교관들은 새롭게 유엔 대사가 되는 사람에게 있어 즉각적인 우려 사항은 세계 기구에서 미국의 지출을 억제하는 것이어야 한다고 말합니다.

